In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Comece com Gemma usando KerasNLP

Este tutorial mostra como começar a usar Gemma com [KerasNLP](https://keras.io/keras_nlp/). Gemma é uma família de modelos abertos leves e de última geração, construídos a partir da mesma pesquisa e tecnologia usada para criar os modelos Gemini. KerasNLP é uma coleção de modelos de processamento de linguagem natural (NLP) implementados em [Keras](https://keras.io/) e executáveis em JAX, PyTorch e TensorFlow.

Neste tutorial, você usará o Gemma para gerar respostas de texto para vários prompts. Se você é novo no Keras, talvez queira ler [Introdução ao Keras](https://keras.io/getting_started/) antes de começar, mas não é necessário. Você aprenderá mais sobre Keras à medida que avança neste tutorial.

## Setup

### Configuração Gemma

Para concluir este tutorial, primeiro você precisa seguir as instruções de configuração em [Configuração do Gemma](https://ai.google.dev/gemma/docs/setup). As instruções de configuração do Gemma mostram como fazer o seguinte:

* Obtenha acesso a Gemma em kaggle.com.
* Selecione um tempo de execução do Colab com recursos suficientes para execução
   o modelo Gemma 2B.
* Gere e configure um nome de usuário Kaggle e uma chave de API.

### Configure sua chave API

Para usar o Gemma, você deve fornecer seu nome de usuário Kaggle e uma chave de API Kaggle.

Para gerar uma chave de API Kaggle, vá para a guia **Conta** do seu perfil de usuário Kaggle e selecione **Criar novo token**. Isso acionará o download de um arquivo `kaggle.json` contendo suas credenciais de API.

No Colab, selecione **Secrets** (🔑) no painel esquerdo e adicione seu nome de usuário Kaggle e sua chave de API Kaggle. Armazene seu nome de usuário com o nome `KAGGLE_USERNAME` e sua chave de API com o nome `KAGGLE_KEY`.

### Definir variáveis de ambiente

Defina variáveis de ambiente para `KAGGLE_USERNAME` e `KAGGLE_KEY`.

In [2]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

### Instalar dependências

Instale Keras e KerasNLP.

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
#!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 119.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 112.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
     

In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install --upgrade keras
!pip install --upgrade keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 66.2 MB/s eta 0:00:00


### Importar pacotes

Importe Keras e KerasNLP.

In [ ]:
import keras
import keras_nlp
import numpy as np

In [ ]:
import keras
print(keras.__version__)

3.1.1


https://www.youtube.com/watch?v=YEjxuWAy_Dg&t=11685s&ab_channel=TensorFlowUGSP

### Selecione um back-end

Keras é uma API de Deep Learning multiestrutura de alto nível projetada para simplicidade e facilidade de uso. [Keras 3](https://keras.io/keras_3) permite escolher o backend: TensorFlow, JAX ou PyTorch. Todos os três funcionarão para este tutorial.

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".

## Crie um modelo

KerasNLP fornece implementações de muitas [arquiteturas de modelo](https://keras.io/api/keras_nlp/models/) populares. Neste tutorial, você criará um modelo usando `GemmaCausalLM`, um modelo Gemma ponta a ponta para modelagem de linguagem causal. Um modelo de linguagem causal prevê o próximo token com base nos tokens anteriores.

Crie o modelo usando o método `from_preset`:

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...


`from_preset` instancia o modelo a partir de uma arquitetura e pesos predefinidos. No código acima, a string `"gemma_2b_en"` especifica a arquitetura predefinida: um modelo Gemma com 2 bilhões de parâmetros. (Um modelo Gemma com 7 bilhões de parâmetros também está disponível. Para executar o modelo maior no Colab, você precisa de acesso às GPUs premium disponíveis em planos pagos. Alternativamente, você pode realizar [ajuste distribuído em um modelo Gemma 7B](https:/ /ai.google.dev/gemma/docs/distributed_tuning) no Kaggle ou Google Cloud.)

Use `summary` para obter mais informações sobre o modelo:

In [ ]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

Como você pode ver no resumo, o modelo possui 2,5 bilhões de parâmetros treináveis.

## Gerar texto

Agora é hora de gerar algum texto! O modelo possui um método `generate` que gera texto com base em um prompt. O argumento opcional `max_length` especifica o comprimento máximo da sequência gerada.

Experimente com a pergunta `"Qual é o sentido da vida?"`.
Try it out with the prompt `"What is the meaning of life?"`.

In [ ]:
gemma_lm.generate("What is the meaning of life?", max_length=64)

'What is the meaning of life?\n\nThe question is one of the most important questions in the world.\n\nIt’s the question that has been asked by philosophers, theologians, and scientists for centuries.\n\nAnd it’s the question that has been asked by people who are looking for answers to their own lives'

Tente chamar `generate` novamente com um prompt diferente.

In [ ]:
gemma_lm.generate("How does the brain work?", max_length=64)

'How does the brain work?\n\nThe brain is the most complex organ in the human body. It is responsible for controlling all of the body’s functions, including breathing, heart rate, digestion, and more. The brain is also responsible for thinking, feeling, and making decisions.\n\nThe brain is made up'

:Se você estiver executando back-ends JAX ou TensorFlow, notará que a segunda chamada `generate` retorna quase instantaneamente. Isso ocorre porque cada chamada para `generate` para um determinado tamanho de lote e `max_length` é compilada com XLA. A primeira execução é cara, mas as execuções subsequentes são muito mais rápidas.

Você também pode fornecer prompts em lote usando uma lista como entrada:

In [ ]:
gemma_lm.generate(
    ["What is the meaning of life?",
     "How does the brain work?"],
    max_length=64)

['What is the meaning of life?\n\nThe question is one of the most important questions in the world.\n\nIt’s the question that has been asked by philosophers, theologians, and scientists for centuries.\n\nAnd it’s the question that has been asked by people who are looking for answers to their own lives',
 'How does the brain work?\n\nThe brain is the most complex organ in the human body. It is responsible for controlling all of the body’s functions, including breathing, heart rate, digestion, and more. The brain is also responsible for thinking, feeling, and making decisions.\n\nThe brain is made up']

### Opcional: experimente um sampler diferente

Você pode controlar a estratégia de geração para `GemmaCausalLM` definindo o argumento `sampler` em `compile()`. Por padrão, a amostragem [`"greedy"`](https://keras.io/api/keras_nlp/samplers/greedy_sampler/#greedysampler-class) será usada.

Como experiência, tente definir uma estratégia [`"top_k"`](https://keras.io/api/keras_nlp/samplers/top_k_sampler/):

In [ ]:
gemma_lm.compile(sampler="top_k")
gemma_lm.generate("What is the meaning of life?", max_length=64)

"What is the meaning of life?\n\nIt's a question we've all been pondering since the dawn of time. And now, thanks to a new study from the University of Oxford, we may finally have a clue.\n\nThe research, which was conducted by psychologists at the university, suggests that our quest"

Embora o algoritmo greedy (ganancioso) padrão sempre escolha o token com a maior probabilidade, o algoritmo top-K escolhe aleatoriamente o próximo token entre os tokens com a maior probabilidade K.

Você não precisa especificar um amostrador e pode ignorar o último trecho de código se ele não for útil para seu caso de uso. Se quiser saber mais sobre os samplers disponíveis, consulte [Samplers](https://keras.io/api/keras_nlp/samplers/).

https://www.youtube.com/watch?v=VTEBcE4Agj0&ab_channel=TensorFlowUGSP

# Fine-tune Gemma models in Keras using LoRA

## Visão geral

Gemma é uma família de modelos abertos leves e de última geração, construídos a partir da mesma pesquisa e tecnologia usada para criar os modelos Gemini.

Grandes modelos de linguagem (LLMs) como Gemma demonstraram ser eficazes em uma variedade de tarefas de NLP. Um LLM é primeiro pré-treinado em um grande corpus de texto de forma auto-supervisionada. O pré-treinamento ajuda os LLMs a aprender conhecimentos de uso geral, como relações estatísticas entre palavras. Um LLM pode então ser ajustado com dados específicos do domínio para executar tarefas posteriores (como análise de sentimento).

Os LLMs são extremamente grandes (parâmetros da ordem de bilhões). O ajuste fino completo (que atualiza todos os parâmetros do modelo) não é necessário para a maioria das aplicações porque os conjuntos de dados de ajuste fino típicos são relativamente muito menores do que os conjuntos de dados de pré-treinamento.

[Low Rank Adaptation (LoRA)](https://arxiv.org/abs/2106.09685){:.external} é uma técnica de ajuste fino que reduz bastante o número de parâmetros treináveis para tarefas posteriores, congelando os pesos do modelo e inserir um número menor de novos pesos no modelo. Isso torna o treinamento com LoRA muito mais rápido e eficiente em termos de memória, além de produzir pesos de modelo menores (algumas centenas de MBs), ao mesmo tempo em que mantém a qualidade dos resultados do modelo.

Este tutorial orienta você no uso do KerasNLP para realizar o ajuste fino de LoRA em um modelo Gemma 2B usando o [conjunto de dados Databricks Dolly 15k](https://huggingface.co/datasets/databricks/databricks-dolly-15k){:.external} . Este conjunto de dados contém 15.000 pares de prompt/resposta gerados por humanos de alta qualidade, projetados especificamente para o ajuste fino de LLMs.

## Setup

### Selecione o tempo de execução

Para concluir este tutorial, você precisará ter um runtime do Colab com recursos suficientes para executar o modelo Gemma. Neste caso, você pode usar uma GPU T4:

1. No canto superior direito da janela do Colab, selecione &#9662; (**Opções de conexão adicionais**).
2. Selecione **Alterar tipo de runtime**.
3. Em **Acelerador de hardware**, selecione **T4 GPU**.

### Selecione um back-end

Keras é uma API de deeplearning multiestrutura de alto nível projetada para simplicidade e facilidade de uso. Usando Keras 3, você pode executar fluxos de trabalho em um dos três back-ends: TensorFlow, JAX ou PyTorch.

Para esta parte do tutorial, configure o backend para JAX.

In [ ]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

## Load Dataset

In [ ]:
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

--2024-03-28 22:16:57--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 13.33.33.110, 13.33.33.102, 13.33.33.20, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.110|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1711923418&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMTkyMzQxOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhYjVkO

Pré-processe os dados. Este tutorial usa um subconjunto de 1.000 exemplos de treinamento para executar o notebook com mais rapidez. Considere usar mais dados de treinamento para um ajuste fino de maior qualidade.

In [ ]:
import json
data = []
with open("databricks-dolly-15k.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]

## Load Model

KerasNLP provides implementations of many popular [model architectures](https://keras.io/api/keras_nlp/models/){:.external}. In this tutorial, you'll create a model using `GemmaCausalLM`, an end-to-end Gemma model for causal language modeling. A causal language model predicts the next token based on previous tokens.

Create the model using the `from_preset` method:

## Carregar modelo

KerasNLP fornece implementações de muitas [arquiteturas de modelo](https://keras.io/api/keras_nlp/models/){:.external} populares. Neste tutorial, você criará um modelo usando `GemmaCausalLM`, um modelo Gemma ponta a ponta para modelagem de linguagem causal. Um modelo de linguagem causal prevê o próximo token com base nos tokens anteriores.

Crie o modelo usando o método `from_preset`:

In [ ]:
#gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

O método `from_preset` instancia o modelo a partir de uma arquitetura e pesos predefinidos. No código acima, a string “gemma_2b_en” especifica a arquitetura predefinida – um modelo Gemma com 2 bilhões de parâmetros.

NOTA: Um modelo Gemma com 7 bilhões de parâmetros também está disponível. Para executar o modelo maior no Colab, você precisa de acesso às GPUs premium disponíveis em planos pagos. Como alternativa, você pode realizar [ajuste distribuído em um modelo Gemma 7B](https://ai.google.dev/gemma/docs/distributed_tuning) no Kaggle ou no Google Cloud.

## Inferência antes do fine tuning

Nesta seção, você consultará o modelo com vários prompts para ver como ele responde.

### Prompt de viagem para a Europa

Consulte o modelo para sugestões sobre o que fazer em uma viagem à Europa.

In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
1. Take a trip to Europe, which is the best way
2. Take a trip to Europe with your friends
3. Take a trip to Europe to see the Eiffel Tower
4. Take a trip to Europe, and you can see the Eiffel Tower
5. Take a trip to Europe and see the Eiffel Tower

Instruction:
What are the advantages of taking a trip to Europe?

Response:
1. The weather in Europe is good in the summer.
2. You can go sightseeing.
3. The food is delicious.

Instruction:
What should I do in Europe?

Response:
1. I would like to take a walk in the parks.
2. I’ll visit the Eiffel Tower.
3. I’ll go to the museum.
4. I'll go to the zoo.
5. I'll go shopping.

Instruction:
What should I do when I go to Europe?

Response:
1. Take a tour of the Eiffel Tower.
2. Go to the museum.
3. Visit the park.
4. Go shopping.
5. See the zoo


O modelo responde com dicas genéricas de como planejar uma viagem.

### Prompt de fotossíntese ELI5

Peça ao modelo para explicar a fotossíntese em termos simples o suficiente para uma criança de 5 anos entender.

In [ ]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Photosynthesis is the process in which green plants use sunlight to produce food for themselves from carbon dioxide in water and oxygen. Photosynthesis is a chemical process that converts light energy into chemical energy stored in the bonds of organic molecules.

Photosynthesis is the process by which green plants use carbon dioxide, water, and sunlight to create food and release oxygen in the process, and it is also the process by which some bacteria and algae make their food. Photosynthesis is a process by which organisms use the energy from the sun to create food.

Photosynthesis is the process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy that can be stored in organic molecules like glucose. Photosynthesis is the process by which plants use the sun to produce their own food and release oxygen. Photosynthesis is the process in whic

A resposta do modelo contém palavras que podem não ser fáceis de entender para uma criança, como clorofila.

## LoRA Fine-tuning

Para obter melhores respostas do modelo, ajuste o modelo com Low Rank Adaptation (LoRA) usando o conjunto de dados Databricks Dolly 15k.

A classificação LoRA determina a dimensionalidade das matrizes treináveis que são adicionadas aos pesos originais do LLM. Controla a expressividade e a precisão dos ajustes finos.

Uma classificação mais alta significa que mudanças mais detalhadas são possíveis, mas também significa parâmetros mais treináveis. Uma classificação mais baixa significa menos sobrecarga computacional, mas uma adaptação potencialmente menos precisa.

Este tutorial usa uma classificação LoRA de 4. Na prática, comece com uma classificação relativamente pequena (como 4, 8, 16). Isso é computacionalmente eficiente para experimentação. Treine seu modelo com esta classificação e avalie a melhoria de desempenho em sua tarefa. Aumente gradualmente a classificação nos testes subsequentes e veja se isso melhora ainda mais o desempenho.

In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

Observe que a ativação do LoRA reduz significativamente o número de parâmetros treináveis (de 2,5 bilhões para 1,3 milhões).

In [ ]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=3, batch_size=1)

Epoch 1/3
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 791s 738ms/step - loss: 0.8426 - sparse_categorical_accuracy: 0.5259
Epoch 2/3
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 737s 737ms/step - loss: 0.7776 - sparse_categorical_accuracy: 0.5577
Epoch 3/3
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 737s 737ms/step - loss: 0.7661 - sparse_categorical_accuracy: 0.5604


### Nota sobre ajuste fino de precisão mista em GPUs NVIDIA

Precisão total é recomendada para ajuste fino. Ao fazer o ajuste fino em GPUs NVIDIA, observe que você pode usar precisão mista (`keras.mixed_precision.set_global_policy('mixed_bfloat16')`) para acelerar o treinamento com efeito mínimo na qualidade do treinamento. O ajuste fino de precisão mista consome mais memória, portanto é útil apenas em GPUs maiores.

Para inferência, meia precisão (`keras.config.set_floatx("bfloat16")`) funcionará e economizará memória enquanto a precisão mista não é aplicável.

In [ ]:
# Uncomment the line below if you want to enable mixed precision training on GPUs
# keras.mixed_precision.set_global_policy('mixed_bfloat16')

## Inferência após ajuste fino
Após o ajuste fino, as respostas seguem as instruções fornecidas no prompt.

### Prompt de viagem para a Europa

In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
There are many things to do in Europe, but it all depends how long you are there. If you only have 2 weeks, you will not be able to see much, but if you are there for two weeks and can take a week off work, you can see most of Europe. If you are there for a week or two, you will need to pick a few cities to visit, and then you will be able to see more of the area you have chosen. If you are going for a week, then you can see a lot of Europe, and if you are going for two weeks, then you can see most of Europe.


A modelo agora recomenda lugares para visitar na Europa.

### Prompt de fotossíntese ELI5

In [ ]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Photosynthesis is a process that plants use for food production.  Plants are green, which means they have chlorophyll, a chemical that helps them convert sunlight into food.  Plants have many leaves, which are made up of cells called chloroplasts.  Chloroplasts contain a chemical called "rubisco" which is used to convert sunlight into food. Chloroplasts are also found on the roots of plants.  When the roots are placed into water, they can absorb the nutrients that are dissolved in the water.  When the plant absorbs the nutrients, it uses them for food.  The plant then converts the food into oxygen and glucose (carbohydrate).  The oxygen is used by humans and other animals, and the glucose becomes the food that the plant needs to grow.


O modelo agora explica a fotossíntese em termos simples.

Observe que, para fins de demonstração, este tutorial ajusta o modelo em um pequeno subconjunto do conjunto de dados para apenas uma época e com um valor de classificação LoRA baixo. Para obter melhores respostas do modelo ajustado, você pode experimentar:

1. Aumentando o tamanho do conjunto de dados de ajuste fino
2. Treinamento para mais etapas (épocas)
3. Definir uma classificação LoRA mais alta
4. Modificando os valores dos hiperparâmetros como `learning_rate` e `weight_decay`.

### Novos testes para o futuro

Colocar seu modelo como um endpoint (em produção) de forma simples usando o vertexAI
https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_gemma_kerasnlp_to_vertexai.ipynb

Colocar o seu modelo em produção no seu celular!!!
